In [4]:
# Import packages
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from datetime import date

use the last 2yrs with a 1d interval to filter stocks, find stop-loss
- try a 59day with short interval to find optimal buy/sell 
- also can try 1d interval to find optimal buy/sell, but less optimistic on this

### look for high short term vol, low long term vol

Will need to be able to download stock data. 
- Limited by data availability. Ideally would be using intraday prices going back 2yrs but beyond 60-days that data isn't available from yfinance. 
Chosen to get recent and longer term price variability (PV) separately. Looking for low on both

In [ ]:
def download_from_ticker(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())

    short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    return [short_data, long_data]


def short_long_price_variability(short_long_list):
    
    """
    Calc short (2mo) and long (2yr) term price variability (price adjusted standard deviation).
    """
    
    short_data = short_long_list[0]
    long_data = short_long_list[1]
    
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
    twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
    twomo_avg = twomo_mid_data.mean() #calculate average 
    twomo_std_dev = twomo_mid_data.std() #calculate deviation
    twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    return twomo_vol, twoyr_vol

In [10]:
#only check long if short matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())

    short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
    twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
    twomo_avg = twomo_mid_data.mean() #calculate average 
    twomo_std_dev = twomo_mid_data.std() #calculate deviation
    twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    
    if ((0.01 < twomo_vol) & (0.015 > twomo_vol)): #only check long data if short data fits, save runtime
        long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
        twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
        twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
        twoyr_avg = twoyr_mid_data.mean()
        twoyr_std_dev = twoyr_mid_data.std()
        twoyr_vol = twoyr_std_dev / twoyr_avg
    else:
        twoyr_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [ ]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.025 < twoyr_vol) & (0.035 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [11]:
# testing on our poster-child, expecting low price variability
print(short_long_price_variability("GUD.TO"))

# expecting pretty high volatility on this one
print(short_long_price_variability("BBBY"))

#works on indices as well
print(short_long_price_variability("^DJI"))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
{'Short PV': 0.013387338973652053, 'Long PV': 0.03030104479822352}
[*********************100%***********************]  1 of 1 completed
{'Short PV': 0.39027473494521814, 'Long PV': -1}
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
{'Short PV': 0.014442321645563024, 'Long PV': 0.04753019040683213}


It looks like BBBY has about 25x short and 20x long PV compared to GUD.TO. So the function is doing what it's supposed to.

DJI has comparable short term PV, and about 1.5x long.

GUD.TO is remarkably boring. 

Retreived ticker names from https://www.nasdaq.com/market-activity/stocks/screener

Filtered out stocks smaller than 300m mktcap, manually combined nasdaq and nyse tickers (total 3800)

Can add Canadian stocks later if desired.

In [12]:
tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                      'rket-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"})

In [ ]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

In [ ]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

In [ ]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

In [ ]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

In [ ]:
#opting to set min variability instead of volume to filter based on liquidity but could change later 
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

PNM is flat in the 2yr, but not enough chop for this 
TXO is a recent IPO, not enough data.
BCSA is not flat, slowly rising. Also a recent (~1yr) IPO so not enough data
DCRD is a SPAC (explains long term PV). Recent PV too crazy (up 47% Feb 13-20)
HONE is great. Exactly what we're looking for (although ideally no dividend)
LANDM looks like it was stable, but now trending downwards in the last 6mo. Not ideal. 

Based on HONE's Long PV of 0.34, going to expand Long PV range to 0.42 to see if I can get more hits slightly higher. Moving min to 0.3 since I've screened below and don't see anything. 

In [13]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.03 < twoyr_vol) & (0.042 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [ ]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

In [ ]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

In [ ]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

In [ ]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

In [ ]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

EQC looked good, but just broke down after a special div payment
JNJ is decent. Larger swings in price but may be enough of them not to rule it out.
CBSH looks good.
CZNC declining 1yr MA not ideal. 
ENLT recent IPO. No
FNLC looks good.
MLYS recent IPO
SKWD recent IPO.

CBSH and FNLC are towards the top end of the Long PV range, may as well check higher. 

In [ ]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.042 < twoyr_vol) & (0.06 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [ ]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

In [ ]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

In [ ]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

In [ ]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

In [ ]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

ADC looks decent. Bigger dif between highs and lows than I'd like.
AEE looks decent. Looks like a general uptrend but not too pronounced
AMCR looks nice. 
AVA is pretty good. Last 1yr better than previous.
BDJ looks to be trending down slightly. 
BDX looks nice. 
BXMX looks decent. Very cyclical around the dividend. Maybe slightly trending down. 
CAG is ok. Bigger dif between highs and lows than I'd like.
CAJ was looking good. Fell off in last 6mo. 
CAPL looks nice. Farthest end of 2yr is uptrend but since then nice. 
CHE looks good. 
CL is ok. Bigger dif between highs and lows than I'd like.
CNA not great. Wide wavelength. 
CP looks great. 
CRBG is recent IPO. No. 
DIAX looks general downtrend. No.
DNP no.
DPG looks pretty good. 
DUK not great. 
EIG looks ok. Something strange big movement in Oct 2022
ES was good. May have broken down recently. 
FCPT is ok. Bigger dif between highs and lows than I'd like
FEI is good. Far end of 2yr rising but flat since. 
GHC is ok. Not great. 
GIB no.
HE pretty good. 
HFRO no. 
HMN slight downtrend. Potentially ok.
HRL no. Strong support but hitting lower highs. 
HTD not great. 
IBM is ok. Bigger dif between highs and lows than I'd like.
IDA is ok. Bigger dif between highs and lows than I'd like.
IGD no. 
INGR not great. 
KMB is ok. Bigger dif between highs and lows than I'd like.
KMI is ok. Bigger dif between highs and lows than I'd like.
LADR is ok. Don't like the big drop in Oct 22.
MMP in a slight uptrend but still looks good. 
MSM is ok. Bigger dif between highs and lows than I'd like.
NID no. 
NNN is ok. Bigger dif between highs and lows than I'd like.
NVS is ok. Bigger dif between highs and lows than I'd like.
O is ok. Don't like the big drop in Oct 22.
OFC is ok. Don't like the big drop in Oct 22.
PB is pretty good. 
PFS is ok. Bigger dif between highs and lows than I'd like.
PM is ok. Bigger dif between highs and lows than I'd like.
POR no. 
PPL looks ok, but don't like that it's hitting lower lows since Nov 21. 
PRS no. 
SCM just broke out. No. 
SON looks pretty good. 
SPH looks pretty good. 
STZ is ok. Bigger dif between highs and lows than I'd like.
SYY is pretty good. Bigger dif between highs and lows than I'd like.
THG no. 
VOYA just broke out. No. 
WEC is ok. Bigger dif between highs and lows than I'd like.
WMT is ok. Bigger dif between highs and lows than I'd like.
WPC no. 
WTRG is ok. Bigger dif between highs and lows than I'd like.
ARCC no. 
AROW no. 
BTWN no volume. 
BWB is ok but looks too long wavelength. 
CAC no. In slight downtrend. 
CATC looks pretty good. 
CATY looks ok. 
CCNE looks pretty good. 
CGBD was flat. Breaking out now. 
CIVB no. Trending down. 
CTBI looks good. 
DCOM looks good. 
DGICA is ok. Bigger dif between highs and lows than I'd like.
EVRG is ok. Bigger dif between highs and lows than I'd like.
FRPH looked a little off far end of 2yr. Very good now. 
GGAA no. SPAC.
GLPI slight uptrend. No. 
GNTY no. 
GSBC good in the last year. 
IEP good for the past year. 
INDB pretty good for the past year. 
KDP decent. 
MDLZ decent. High amplitude. 
MGEE no. 
MOFG pretty good. 
MSBI pretty good. 
NMFC no. Was good but now too big amp. 
NWBI no. Way too big amp. 
OBIO no. Spac?
OBNK decent. Maybe slight downtrend. 
OCSL no. 
REYN big amplitude. Prob not. 
RPRX big aplitude. Prob not. 
SAFT no. Slight uptrend, large amp. 
SIRI would have been great but just crashed. 
TCPC decent. Slight downtrend. 
THFF is ok. 
TOWN is great. 
VGAS no. SAPC. 
VLGEA is decent. Don't like the big drop in Oct 22.
WABC good in the past 1.5yr.
WAFD is ok. 
WALD no. SPAC.
XEL not great. High amplitude. 

Looks like the upper end of Long PV from this batch are generally not great (with a couple exceptions), meaning we've probably found our upper limit. 

The ones that look really good from this are:
      Short     Long
HONE  0.015847  0.034489
CBSH  0.018631  0.041063
FNLC  0.015954  0.040111
AMCR  0.029136  0.043482
BDX   0.023021  0.043537
CAPL  0.036246  0.056862 
CHE   0.016904  0.046703
CP    0.019901  0.053180
CTBI  0.038825  0.048464
DCOM  0.030092  0.059724
FRPH  0.023382  0.050308
GSBC  0.016853  0.046868
IEP   0.017944  0.054363
INDB  0.023662  0.059701
TOWN  0.016220  0.050351
WABC  0.019999  0.048959

Next model prob want 0.3 < Long PV < 0.55 & 

Can use optimization to figure out buys (at least initially?)

In [36]:
best_stocks = ["GUD.TO", "HONE", "CBSH", "FNLC", "AMCR", "BDX", "CAPL", "CHE", "CP", "CTBI", "DCOM", "FRPH", "GSBC", "IEP", 
               "INDB", "TOWN", "WABC"]

start_date = datetime.date(2023,3,1) - datetime.timedelta(730) 
end_date = datetime.date(2023,3,1)

In [44]:
#real working one

for i, stock in enumerate(best_stocks):
    info = yf.download(stock, start=start_date, end=end_date)[["High", "Low"]]
    
    info = info.round(2)
    
    owned = False
    max_profit = 0.
    best_buy = 0.
    best_sell = 0.

    min_low = np.min(info["Low"])
    max_high = np.max(info["High"])
    
    #incrementing at 1% of buy price to reduce runtime
    #ideally would do 0.01 for all but that's probably more trouble than it's worth at high prices
    increment = min_low/100
    
    buy_prices = np.arange(min_low, max_high, increment)
    
    def buy(buy_price):
        #need to add these globals since vectorizing, otherwise the program complains that it's calling  
        #.  variables before they're defined
        global owned
        global max_profit
        global best_sell
        global best_buy
        
        sell_prices = np.arange(buy_price, max_high, increment)
        
        def sell(sell_price):
            global owned
            global max_profit
            global best_sell
            global best_buy
            
            profit = 0.
            
            for index, row in info.iterrows():
                if (owned):
                    if (row["High"] >= sell_price):
                        profit += (sell_price - buy_price)
                        owned = False
                else:
                    if (row["Low"] <= buy_price):
                        owned = True
                
                if (profit > max_profit):
                    max_profit = round(profit, 2)
                    best_sell = round(sell_price, 2)
                    best_buy = round(buy_price, 2)
        
        sell_vec = np.vectorize(sell)
        sell_vec(sell_prices)
                    
    buy_vec = np.vectorize(buy)
    buy_vec(buy_prices)
    
    annual_return = max_profit / (best_buy*2) * 100
    
    print(f"The max profit for {stock} is {max_profit} at buy price: {best_buy} and sell price: {best_sell}")
    print(f"That equates to a {annual_return}% annual return")


[*********************100%***********************]  1 of 1 completed
The max profit for GUD.TO is 3.66 at buy price: 5.22 and sell price: 5.47
That equates to a 35.05747126436782% annual return
[*********************100%***********************]  1 of 1 completed
The max profit for HONE is 10.87 at buy price: 13.53 and sell price: 14.13
That equates to a 40.169992609016994% annual return
[*********************100%***********************]  1 of 1 completed
The max profit for CBSH is 46.36 at buy price: 67.99 and sell price: 71.08
That equates to a 34.09324900720694% annual return
[*********************100%***********************]  1 of 1 completed
The max profit for FNLC is 23.54 at buy price: 28.66 and sell price: 29.68
That equates to a 41.06769016050244% annual return
[*********************100%***********************]  1 of 1 completed
The max profit for AMCR is 6.2 at buy price: 11.16 and sell price: 12.71
That equates to a 27.77777777777778% annual return
[*********************100%*

Next steps: decide on stop loss
- drops X% below buy-price?
    - different for each stock or constant X?
        - ideally they have similar variability so can use a constant
- drops below min_low?
    - maybe better, that way we have defined the range in which we think we can predict prices, and if it 
      leaves that range we discard


In [46]:
info = yf.download("HONE", start=start_date, end=end_date)[["High", "Low"]]

min_low = info.Low.min()
max_high = info.High.max()
    
for date in info.keys:
    if (owned):
        if (info.High >= sell_price):
            profit += sell_price - buy_price
            owned = False
    else:
          if (info.Low <= buy_price):
            owned = True
    print(profit)

[*********************100%***********************]  1 of 1 completed


TypeError: 'method' object is not iterable

In [ ]:
import yfinance as yf #contains stock price data
import numpy as np
import pendulum 
import matplotlib.pyplot as plt 

stock_info = yf.Ticker('TSLA').info

market_price = stock_info['regularMarketPrice']
previous_close_price = stock_info['regularMarketPreviousClose']
print('market price ', market_price)
print('previous close price ', previous_close_price)

price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   interval='1mo', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   actions=False)
time_series = list(price_history['Open'])
dt_list = [pendulum.parse(str(dt)).float_timestamp for dt in list(price_history.index)]
plt.style.use('dark_background')
plt.plot(dt_list, time_series, linewidth=2)

In [ ]:
def short_long_returns_volatility_ratio(short_long_list):
    
    """
    Calc the ratio between short (2mo) and long (2yr) term daily volatility of log returns.
    Higher output means high short term vol relative to long term.
    
    Literature uses log returns to calculate volatility. For my purposes I'm not sure this is necessary.
    I only care about deviation in price so a more simple approach should suffice. 
    """
    
    short_data = short_long_list[0]
    long_data = short_long_list[1]
    
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twomo_log_returns = twomo_midpoint.assign(log_returns = lambda x: np.log(x.Mid / x.Mid.shift()))
    twomo_std_dev = np.array(twomo_log_returns["log_returns"])[1:].std()
    twomo_vol = twomo_std_dev * (59**0.5) 
    
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2)
    twoyr_log_returns = twoyr_midpoint.assign(log_returns = lambda x: np.log(x.Mid / x.Mid.shift()))
    twoyr_std_dev = np.array(twoyr_log_returns["log_returns"])[1:].std()
    twoyr_vol = twoyr_std_dev * (730**0.5)
    
    return twomo_vol, twoyr_vol